In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import numpy as np
import rasterio
from tqdm import tqdm

from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import Labeler
from utils.filter import (
    has_cloud, is_too_black2, mask_full_cloud, patch_nocloud
)


In [90]:
import os
os.chdir("../src")
os.getcwd()

In [ ]:
from train_pipeline_utils.download_data import load_pleiade_data
from classes.data.satellite_image import SatelliteImage

In [ ]:
load_pleiade_data(2022, "972")

In [ ]:
load_pleiade_data(2017, "972")

In [79]:
load_pleiade_data(2017, "976")

In [ ]:
load_pleiade_data(2022, "973")

In [80]:
load_pleiade_data(2020, "976")

In [ ]:
from rasterio.errors import RasterioIOError

In [ ]:
from tqdm import tqdm

In [78]:
file_path = '../data/PLEIADES/2022/GUYANE'
output_directory_name = '../nuagespleiades/PLEIADES/2022/GUYANE'

output_masks_path = output_directory_name

if os.path.exists(output_masks_path):
    print("fichiers déjà écrits")

if not os.path.exists(output_masks_path):
    os.makedirs(output_masks_path)


list_name = os.listdir(file_path)
list_path = [file_path + "/" + name for name in list_name]


for path, file_name in tqdm(zip(list_path, list_name), total=len(list_path), desc='Processing'):
    try:
        big_satellite_image = SatelliteImage.from_raster(
            file_path=path, dep=None, date=None, n_bands=3
        )
        
    except RasterioIOError:
        continue

    else:
        boolean = has_cloud(big_satellite_image)

        if boolean:
            mask_full = mask_full_cloud(big_satellite_image)
            file_name = file_name.split(".")[0]
            np.save(output_masks_path + "/" + file_name + ".npy", mask_full)

In [ ]:
list_nuages = os.listdir(output_directory_name)
len(list_nuages)

In [99]:
from utils.utils import update_storage_access

In [108]:
update_storage_access()
fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}
)
source_folder = '../nuagespleiades/PLEIADES/2020'
s3_destination = 's3/projet-slums-detection/Donnees/NUAGESPLEIADES/2020'
fs.put(source_folder, s3_destination, recursive=True)

In [103]:
os.getcwd()

In [109]:
! export MC_HOST_s3=https://5UP3C4JH842UU09WUXCP:w6DbxVTDBgryZEGtRPF2chzcfLc+ARFwnZA5YMB4:eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiI1VVAzQzRKSDg0MlVVMDlXVVhDUCIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNjgzODI4MDA5LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6InJheWEuYmVyb3ZhQGVsZXZlLmVuc2FpLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTY4MzkxNTMwMCwiZmFtaWx5X25hbWUiOiJCRVJPVkEiLCJnaXZlbl9uYW1lIjoiUmF5YSIsImdyb3VwcyI6WyJmdW5hdGhvbiIsInNsdW1zLWRldGVjdGlvbiJdLCJpYXQiOjE2ODM4Mjg4OTgsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiI1NDBiYWJhZi1iOTMyLTQxOTUtYTljYS01NTRhYzJjYjYwMjQiLCJsb2NhbGUiOiJlbiIsIm5hbWUiOiJSYXlhIEJFUk9WQSIsIm5vbmNlIjoiZjZjNmZiMjYtNTczZi00NTY4LTg1YzctNjI1NjhkZTNiNzk2IiwicG9saWN5Ijoic3Rzb25seSIsInByZWZlcnJlZF91c2VybmFtZSI6ImlkMTc1MyIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2Vzc2lvblBvbGljeSI6ImV5SldaWEp6YVc5dUlqb2lNakF4TWkweE1DMHhOeUlzSWxOMFlYUmxiV1Z1ZENJNlczc2lSV1ptWldOMElqb2lRV3hzYjNjaUxDSkJZM1JwYjI0aU9sc2ljek02S2lKZExDSlNaWE52ZFhKalpTSTZXeUpoY200NllYZHpPbk16T2pvNmNISnZhbVYwTFhOc2RXMXpMV1JsZEdWamRHbHZiaUlzSW1GeWJqcGhkM002Y3pNNk9qcHdjbTlxWlhRdGMyeDFiWE10WkdWMFpXTjBhVzl1THlvaVhYMHNleUpGWm1abFkzUWlPaUpCYkd4dmR5SXNJa0ZqZEdsdmJpSTZXeUp6TXpwTWFYTjBRblZqYTJWMElsMHNJbEpsYzI5MWNtTmxJanBiSW1GeWJqcGhkM002Y3pNNk9qb3FJbDBzSWtOdmJtUnBkR2x2YmlJNmV5SlRkSEpwYm1kTWFXdGxJanA3SW5Nek9uQnlaV1pwZUNJNkltUnBabVoxYzJsdmJpOHFJbjE5ZlN4N0lrVm1abVZqZENJNklrRnNiRzkzSWl3aVFXTjBhVzl1SWpwYkluTXpPa2RsZEU5aWFtVmpkQ0pkTENKU1pYTnZkWEpqWlNJNld5SmhjbTQ2WVhkek9uTXpPam82S2k5a2FXWm1kWE5wYjI0dktpSmRmVjE5Iiwic2Vzc2lvbl9zdGF0ZSI6ImUzYjQ1MzA3LWMxZTEtNDU3MC04MmZiLTljMGI0NWNhZDcwYiIsInNpZCI6ImUzYjQ1MzA3LWMxZTEtNDU3MC04MmZiLTljMGI0NWNhZDcwYiIsInN1YiI6ImRiNWM5MTg3LWUwNjItNDIyMy04N2UwLTk2YTc5ODYzMDZlOSIsInR5cCI6IkJlYXJlciJ9.TrOIMPmQkwyHMNIxkV3OMdktEvmHuqWwGYNFpBNGvf_5EqvDMtkMR84J52USKE9GJu9xNS_1ca1CHD26tj8AZA@minio.lab.sspcloud.fr

In [111]:
!mc cp --recursive ../nuagespleiades/PLEIADES/2020 s3/projet-slums-detection/Donnees/NUAGESPLEIADES/2020

In [114]:
import gc
import os
import sys
from datetime import datetime
from tqdm import tqdm
import mlflow
import numpy as np
import pytorch_lightning as pl
import torch
import yaml
from pytorch_lightning.callbacks import (
    EarlyStopping,
    LearningRateMonitor,
    ModelCheckpoint,
)
from torch.utils.data import DataLoader
from yaml.loader import SafeLoader

from classes.data.satellite_image import SatelliteImage
from classes.labelers.labeler import BDTOPOLabeler, RILLabeler
from classes.optim.losses import CrossEntropySelfmade
from torch.nn import CrossEntropyLoss
from classes.optim.optimizer import generate_optimization_elements
from data.components.dataset import PleiadeDataset
from models.components.segmentation_models import DeepLabv3Module
from models.segmentation_module import SegmentationModule
from train_pipeline_utils.download_data import load_satellite_data, load_donnees_test
from train_pipeline_utils.handle_dataset import (
    generate_transform,
    select_indices_to_split_dataset
)

from train_pipeline_utils.prepare_data import(
    filter_images,
    label_images,
    save_images_and_masks,
    check_labelled_images
)

from classes.data.satellite_image import SatelliteImage
from classes.data.labeled_satellite_image import SegmentationLabeledSatelliteImage
from utils.utils import update_storage_access, split_array, remove_dot_file
from rasterio.errors import RasterioIOError
from classes.optim.evaluation_model import evaluer_modele_sur_jeu_de_test_segmentation_pleiade
from run_training_pipeline import *

In [ ]:
# Open the file and load the file
with open("../config.yml") as f:
    config = yaml.load(f, Loader=SafeLoader)

list_data_dir, list_masks_cloud_dir, test_dir = download_data(config)

# list_data_dir = ["../data/PLEIADES/2022/MARTINIQUE"]
# list_masks_cloud_dir = ["../data/NUAGESPLEIADES/2022/MARTINIQUE"]

list_output_dir = prepare_train_data(config, list_data_dir, list_masks_cloud_dir)
prepare_test_data(config, test_dir)

model = instantiate_model(config)

train_dl, valid_dl, test_dl = instantiate_dataloader(
    config, list_output_dir
)